In [1]:
import os

In [2]:
%pwd

'/home/tinashe/codes/chicken2/coccidiosis_identification/research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'/home/tinashe/codes/chicken2/coccidiosis_identification'

In [5]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class TrainingConfig:
    root_dir: Path
    trained_model_path: Path
    updated_base_model_path: Path
    training_data: Path
    parameter_epochs: int
    parameter_batch_size: int
    parameter_is_augmentation: bool
    parameter_image_size: list


@dataclass(frozen=True)
class PrepareCallbacksConfig:
    root_dir: Path
    tensorboard_root_log_dir: Path
    checkpoint_model_filepath:Path
    

In [6]:
from classifier.constants import *
from classifier.utils.common_functions import read_yaml, create_dirctories
import tensorflow as tf 

2024-12-06 17:45:32,038 - classifier - INFO - welcome tinashe


2024-12-06 17:45:35.547436: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE4.1 SSE4.2, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [10]:
class ConfigurationManager:
    def __init__(
            self,
            config_filepath= CONFIG_FILE_PATH,
            ):
        parameters_filepath= PARAMETERS_FILE_PATH
        self.config= read_yaml(config_filepath)
        self.parameter= read_yaml(parameters_filepath)
        create_dirctories(self.config.artifacts_root)


    def get_prepare_callbacks_config(self)-> PrepareCallbacksConfig:
        config=self.config.prepare_callbacks
        model_ckpt_dir= os.path.dirname(config.checkpoint_model_filepath)

        create_dirctories([
            Path(model_ckpt_dir),
            Path(config.tensorboard_root_log_dir)
        ])

        prepare_callbacks_config= PrepareCallbacksConfig(
            root_dir= Path(config.root_dir),
            tensorboard_root_log_dir=Path(config.tensorboard_root_log_dir),
            checkpoint_model_filepath=Path(config.checkpoint_model_filepath)
        )
        return prepare_callbacks_config
    

    #initialising training configuration 
    def get_training_config(self)->TrainingConfig:
        training= self.config.training
        prepare_base_model= self.config.prepare_base_model
        parameter= self.parameter

        training_data=os.path.join(self.config.data_ingestion.unzip_dir,"Chicken-fecal-images")
        create_dirctories([Path(training.root_dir)])

        training_config=TrainingConfig(
            root_dir=Path(training.root_dir),
            trained_model_path=Path(training.trained_model_path),
            updated_base_model_path= Path(prepare_base_model.updated_base_model_path),
            training_data= Path(training_data),
            parameter_epochs= parameter.EPOCHS,
            parameter_batch_size= parameter.BATCH_SIZE,
            parameter_is_augmentation= parameter.AUGMENTATION,
            parameter_image_size=parameter.IMAGE_SIZE
       
        )
        return training_config

    



In [11]:
#creating componets
import time